In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [4]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (264.44 seconds): Collecting URLs... indiatimes          

n_total     25760
n_unique    25693
dtype: int64

,pubs,actual_pub
https://abcnews.go.com/Politics/top-diplomat-ukraine-crazy-withhold-security-sasistance-political/story?id=66039011,"abcnews, americanconservative",abcnews
https://axios.com/trump-blamed-rick-perry-call-ukraine-zelensky-8178447a-0374-4ac6-b321-a9454b0565d4.html,"dailybeast, axios",axios
https://businessinsider.co.za/chicken-lickens-secret-menu-comes-in-a-black-box-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/india-pakistan-nuclear-war-death-famine-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/le-creuset-star-wars-collection-2019-10-2,"businessinsider, news24",businessinsider
https://cnn.com/style/article/banksy-devolved-parliament-auction-final-sale-price/index.html,"cnn, axios",cnn
https://hotair.com/archives/jazz-shaw/2019/10/05/universal-basic-income-test-work,"townhall, hotair",hotair
https://hotair.com/archives/karen-townsend/2019/10/05/unhinged-biden-snaps-reporter-focus-man,"townhall, hotair",hotair
https://msnbc.com/ali-velshi/watch/house-democrats-seek-ukraine-documents-from-pence-70641733767,"msnbc, nbcnews",msnbc
https://msnbc.com/am-joy/watch/house-democrats-seek-ukraine-documents-from-pence-70703685916,"msnbc, nbcnews",msnbc


huffpost                  68
cnn                      831
investing.com            923
politico                 239
time                      18
cnbc                     163
fox                      332
bbc                      282
businessinsider          553
morningstar               95
wsj                      106
nytimes                  101
guardian                 151
reuters                 1151
washingtontimes          461
washingtonpost           128
cbs                      402
marketwatch              131
atlantic                 108
vice                      95
npr                      858
newrepublic               21
yahoo                    370
independent              726
heritage                 288
zdnet                    383
townhall                 599
abcnews                   99
hotair                    85
cbc                      301
                        ... 
metro                    243
msnbc                    121
nationalreview           732
news24        

In [9]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [5]:
select_results = crawler.select(); display(*select_results)

100.00% (59.01 seconds): Selecting URLs... cryptonews          

n_total     171
n_unique    171
dtype: int64

,pubs,actual_pub


huffpost                 1
cnn                      1
investing.com           22
politico                 0
time                     0
cnbc                     0
fox                      1
bbc                      2
businessinsider          4
morningstar              0
wsj                      0
nytimes                  1
guardian                11
reuters                 16
washingtontimes          2
washingtonpost           0
cbs                      0
marketwatch              0
atlantic                 0
vice                     2
npr                      0
newrepublic              0
yahoo                   23
independent              4
heritage                 0
zdnet                    2
townhall                 1
abcnews                  0
hotair                   0
cbc                      0
                        ..
metro                    0
msnbc                    0
nationalreview           0
news24                   8
techcrunch               0
arstechnica              0
e

In [10]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [6]:
download_results = crawler.download(); download_results

100.00% (183.16 seconds): Downloading... investing.com       

,downloaded,trashed
abcnews,0,0
afr,1,0
aljazeera,1,9
americanconservative,0,0
arirang,0,0
arstechnica,0,0
arynews,1,1
atlantic,0,0
axios,0,0
bbc,1,1


In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']